In [ ]:
# Instalaçaõ duckdb mlflow plotly
!pip install duckdb mlflow plotly


# Analise Exploratória


In [ ]:
# Bibliotecas
import pandas as pd
import numpy as np
import duckdb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit


In [ ]:
# Carregando dados
consumo = pd.read_csv('/content/consumo.csv')
clima = pd.read_csv('/content/clima.csv')
clientes = pd.read_csv('/content/clientes.csv')

In [ ]:
# Verificando dados
for df, name in zip([consumo, clima, clientes],
                    ['consumo', 'clima', 'clientes']):
    print(f"\n{name.upper()}")
    print(df.shape)
    print(df.dtypes)
    print(df.head())


In [ ]:
# Verificando tipo de dados
consumo['date'] = pd.to_datetime(consumo['date'])
clima['date'] = pd.to_datetime(clima['date'])
print(consumo.dtypes)
print(clima.dtypes)


In [ ]:
df_sorted = consumo.sort_values(['client_id','date'])
df_sorted['lag_1'] = df_sorted.groupby('client_id')['consumption_kwh'].shift(1)

df_sorted[['consumption_kwh','lag_1']].corr()


### Identificou-se forte dependência autoregressiva (ρ ≈ 0.64), indicando que modelos que incorporam histórico recente tendem a apresentar maior capacidade preditiva.

In [ ]:
df = consumo.merge(clientes, on='client_id')
df = df.merge(clima, on=['region','date'])

df[['consumption_kwh','temperature','humidity']].corr()


### Embora a correlação linear com temperatura seja baixa, variáveis climáticas foram mantidas como potenciais efeitos não lineares no modelo.



In [ ]:
consumo_cliente = consumo.groupby('client_id')['consumption_kwh'].mean()

print("Desvio padrão das médias:", consumo_cliente.std())
print("Mínimo:", consumo_cliente.min())
print("Máximo:", consumo_cliente.max())


### Observa-se significativa heterogeneidade estrutural entre clientes, com médias variando entre 9.4 e 19.4 kWh. O desvio padrão entre clientes (~3 kWh) é comparável à variabilidade diária da série, indicando que o perfil individual é um dos principais determinantes do consumo.

In [ ]:
# Missing values
clima = clima.sort_values(['region','date'])

clima['temperature'] = clima.groupby('region')['temperature'].transform(
    lambda x: x.interpolate(method='linear')
)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(6,3))
sns.heatmap(clima.isnull(), cbar=False)
plt.title("Mapa de Missing - Clima")
plt.show()




### A variável temperatura apresenta aproximadamente 5% de valores ausentes, distribuídos de forma esparsa ao longo da série, sem evidência de falhas estruturais concentradas por região ou período.

In [ ]:
clima = clima.sort_values(['region','date'])

clima['temperature'] = clima.groupby('region')['temperature'].transform(
    lambda x: x.interpolate(method='linear')
)

print("Missing após tratamento:", clima['temperature'].isnull().sum())


In [ ]:
clima['temperature'] = clima.groupby('region')['temperature'].transform(
    lambda x: x.fillna(method='bfill').fillna(method='ffill')
)


In [ ]:
clientes['region'].value_counts()


### A região é necessária para integração com dados climáticos e representa apenas 5% da base, opto por exclusão desses clientes para preservar consistência estrutural dos dados.

In [ ]:
clientes = clientes[clientes['region'] != 'Desconhecida']
df = consumo.merge(clientes, on='client_id', how='inner')
df = df.merge(clima, on=['region','date'], how='left')



In [ ]:
print("Shape final:", df.shape)


In [ ]:
print(df.isnull().sum())


In [ ]:
df['client_id'].nunique()


In [ ]:
# Feature Engineering

df = df.sort_values(['client_id','date'])


In [ ]:
# Criar Features Autoregressivas - Principal Driver

df['lag_1'] = df.groupby('client_id')['consumption_kwh'].shift(1)
df['lag_7'] = df.groupby('client_id')['consumption_kwh'].shift(7)


In [ ]:
# Criar Rolling Mean - Suavização
df['rolling_mean_7'] = df.groupby('client_id')['consumption_kwh']\
                           .transform(lambda x: x.rolling(7).mean())


In [ ]:
df['rolling_std_7'] = df.groupby('client_id')['consumption_kwh']\
                          .transform(lambda x: x.rolling(7).std())


In [ ]:
# Features temporais
df['temp_squared'] = df['temperature'] ** 2



In [ ]:
print(df.shape)
print(df.isnull().sum())


In [ ]:
df_model = df.dropna().copy()


In [ ]:
# Columns
print(df_model.columns)


In [ ]:
df_model = df.dropna().copy()


In [ ]:
df_model['date'] = pd.to_datetime(df_model['date'])


In [ ]:
# Features
features = [
    'lag_1',
    'lag_7',
    'rolling_mean_7',
    'rolling_std_7',
    'temperature',
    'humidity',
    'temp_squared',
    'month',
    'dayofweek'
]


In [ ]:
target = 'consumption_kwh'


In [ ]:
df_model['month'] = df_model['date'].dt.month
df_model['dayofweek'] = df_model['date'].dt.dayofweek

In [ ]:
print(df_model.dtypes)


In [ ]:
# Split Temporal
cutoff_date = '2023-06-01'

train = df_model[df_model['date'] < cutoff_date]
test = df_model[df_model['date'] >= cutoff_date]

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


# Modelagem

In [ ]:
# Modelo 1 — Baseline (Linear Regression)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

lr = LinearRegression()
lr.fit(X_train, y_train)

pred_lr = lr.predict(X_test)

mae_lr = mean_absolute_error(y_test, pred_lr)
rmse_lr = np.sqrt(mean_squared_error(y_test, pred_lr))

print("Linear Regression")
print("MAE:", mae_lr)
print("RMSE:", rmse_lr)


### O modelo linear apresentou erro médio absoluto de aproximadamente 1.66 kWh, representando cerca de 11% do consumo médio diário, indicando capacidade preditiva relevante mesmo sob hipótese linear simples.

In [ ]:
# Modelo 2 — Random Forest
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    random_state=42
)

rf.fit(X_train, y_train)

pred_rf = rf.predict(X_test)

mae_rf = mean_absolute_error(y_test, pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, pred_rf))

print("\nRandom Forest")
print("MAE:", mae_rf)
print("RMSE:", rmse_rf)


### O modelo não linear (Random Forest) não apresentou ganho significativo em relação ao modelo linear, sugerindo que a estrutura do problema é majoritariamente linear e dominada por efeitos autoregressivos.

In [ ]:
import pandas as pd

importances = pd.Series(rf.feature_importances_, index=features)
print(importances.sort_values(ascending=False))


### A variável rolling_mean_7 concentra aproximadamente 88% da importância preditiva no modelo não linear, evidenciando que o consumo recente é o principal determinante do consumo futuro, com influência climática marginal.

# Conectar ao DuckDB

In [ ]:
import duckdb
con = duckdb.connect(database='aquarela.db', read_only=False)


In [ ]:
# Criar tabelas
con.register("df_consumo", consumo)
con.register("df_clientes", clientes)
con.register("df_clima", clima)


In [ ]:
con.execute("CREATE OR REPLACE TABLE consumo AS SELECT * FROM df_consumo")
con.execute("CREATE OR REPLACE TABLE clientes AS SELECT * FROM df_clientes")
con.execute("CREATE OR REPLACE TABLE clima AS SELECT * FROM df_clima")


In [ ]:
# Conferindo tabelas
con.execute("SHOW TABLES").fetchall()


In [ ]:
# Criando integrando DuckDB
query = """
CREATE OR REPLACE TABLE consumo_integrado AS
SELECT
    c.client_id,
    c.date,
    c.consumption_kwh,
    cli.region,
    cl.temperature,
    cl.humidity
FROM consumo c
JOIN clientes cli
    ON c.client_id = cli.client_id
JOIN clima cl
    ON cli.region = cl.region
   AND c.date = cl.date
"""
con.execute(query)


In [ ]:
# Conferindo
con.execute("SELECT COUNT(*) FROM consumo_integrado").fetchall()


In [ ]:
#(Window function) Feature via SQL
query_features = """
CREATE OR REPLACE TABLE consumo_features AS
SELECT *,
       LAG(consumption_kwh, 1) OVER (
           PARTITION BY client_id
           ORDER BY date
       ) AS lag_1,

       AVG(consumption_kwh) OVER (
           PARTITION BY client_id
           ORDER BY date
           ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
       ) AS rolling_mean_7

FROM consumo_integrado
"""
con.execute(query_features)


# Pre processamento

In [ ]:
!ls


In [ ]:
%%writefile src/preprocessing.py

import duckdb


def create_integrated_table(db_path="aquarela.db"):
    con = duckdb.connect(database=db_path, read_only=False)

    query = """
    CREATE OR REPLACE TABLE consumo_integrado AS
    SELECT
        c.client_id,
        c.date,
        c.consumption_kwh,
        cli.region,
        cl.temperature,
        cl.humidity
    FROM consumo c
    JOIN clientes cli
        ON c.client_id = cli.client_id
    JOIN clima cl
        ON cli.region = cl.region
       AND c.date = cl.date
    """

    con.execute(query)
    con.close()


def create_feature_table(db_path="aquarela.db"):
    con = duckdb.connect(database=db_path, read_only=False)

    query_features = """
    CREATE OR REPLACE TABLE consumo_features AS
    SELECT *,
           LAG(consumption_kwh, 1) OVER (
               PARTITION BY client_id
               ORDER BY date
           ) AS lag_1,

           AVG(consumption_kwh) OVER (
               PARTITION BY client_id
               ORDER BY date
               ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
           ) AS rolling_mean_7

    FROM consumo_integrado
    """

    con.execute(query_features)
    con.close()


In [ ]:
def load_data():
    consumo = pd.read_csv("consumo.csv")
    clima = pd.read_csv("clima.csv")
    clientes = pd.read_csv("clientes.csv")

    consumo["date"] = pd.to_datetime(consumo["date"])
    clima["date"] = pd.to_datetime(clima["date"])

    return consumo, clima, clientes


In [ ]:
%%writefile src/preprocessing.py

import duckdb


def create_integrated_table(db_path, consumo_df, clientes_df, clima_df):
    con = duckdb.connect(database=db_path, read_only=False)

    # Register the pandas DataFrames as temporary views in the current connection
    con.register("consumo", consumo_df)
    con.register("clientes", clientes_df)
    con.register("clima", clima_df)

    query = """
    CREATE OR REPLACE TABLE consumo_integrado AS
    SELECT
        c.client_id,
        c.date,
        c.consumption_kwh,
        cli.region,
        cl.temperature,
        cl.humidity
    FROM consumo c
    JOIN clientes cli
        ON c.client_id = cli.client_id
    JOIN clima cl
        ON cli.region = cl.region
       AND c.date = cl.date
    """

    con.execute(query)
    con.close()


def create_feature_table(db_path="aquarela.db"):
    con = duckdb.connect(database=db_path, read_only=False)

    query_features = """
    CREATE OR REPLACE TABLE consumo_features AS
    SELECT *,
           LAG(consumption_kwh, 1) OVER (
               PARTITION BY client_id
               ORDER BY date
           ) AS lag_1,

           AVG(consumption_kwh) OVER (
               PARTITION BY client_id
               ORDER BY date
               ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
           ) AS rolling_mean_7

    FROM consumo_integrado
    """

    con.execute(query_features)
    con.close()

from src.preprocessing import create_integrated_table, create_feature_table

# Pass the global DataFrames to the function
create_integrated_table(db_path="aquarela.db", consumo_df=consumo, clientes_df=clientes, clima_df=clima)
create_feature_table()


In [ ]:
%%writefile src/train.py

import duckdb
import pandas as pd
import numpy as np
import joblib
import json
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error


def train_model(db_path="aquarela.db"):

    con = duckdb.connect(database=db_path, read_only=False)
    df = con.execute("SELECT * FROM consumo_features").fetchdf()
    con.close()

    df = df.dropna()
    df["date"] = pd.to_datetime(df["date"])

    features = ["lag_1", "rolling_mean_7", "temperature", "humidity"]
    target = "consumption_kwh"

    cutoff_date = "2023-06-01"
    train = df[df["date"] < cutoff_date]
    test = df[df["date"] >= cutoff_date]

    X_train = train[features]
    y_train = train[target]

    X_test = test[features]
    y_test = test[target]

    model = LinearRegression()
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)

    mae = mean_absolute_error(y_test, predictions)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))

    metrics = {
        "MAE": float(mae),
        "RMSE": float(rmse)
    }

    joblib.dump(model, "models/model_v1.pkl")

    with open("models/metrics_v1.json", "w") as f:
        json.dump(metrics, f)

    return metrics


In [ ]:
!mkdir -p src
!mkdir -p models

In [ ]:
!cat src/preprocessing.py


In [ ]:
# Confirmar existencia de tabelas
import duckdb

con = duckdb.connect("aquarela.db")
print(con.execute("SHOW TABLES").fetchall())


In [ ]:
from src.train import train_model

metrics = train_model()
print(metrics)


In [ ]:
!ls models
